In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv


In [6]:
stroke_data = '/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv'
my_data = pd.read_csv(stroke_data)
my_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [7]:
from sklearn.model_selection import train_test_split

y = my_data.stroke

stroke_predictors = my_data.drop(['stroke'], axis=1)
X = stroke_predictors.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

In [5]:
X_train.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi
3029,28932,36.0,0,0,67.29,36.7
2277,37150,34.0,0,0,83.53,48.5
3002,71669,60.0,0,0,65.16,30.8
246,27153,75.0,0,0,78.80,29.3
2825,58235,76.0,0,0,58.65,25.6


In [22]:
#Model

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 50, random_state=1)
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    mean_score = mean_absolute_error(y_valid, pred)
    return mean_score

In [24]:
new_X_train.head()

,id,age,hypertension,heart_disease,avg_glucose_level
3029,28932,36.0,0,0,67.29
2277,37150,34.0,0,0,83.53
3002,71669,60.0,0,0,65.16
246,27153,75.0,0,0,78.80
2825,58235,76.0,0,0,58.65


In [23]:
#dealing with missing values 
#approach 1 --- drop columns with missing values

col_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

new_X_train = X_train.drop(col_missing, axis=1)
new_X_valid = X_valid.drop(col_missing, axis=1)

print('Approach 1 Mean Score:')
print(score_dataset(new_X_train, new_X_valid, y_train, y_valid))

Approach 1 Mean Score:
0.09025440313111545


In [29]:
#Approach 2 -- Simple Imputation

from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print('Approach 2 Mean Score:')
print(score_dataset(imputed_X_train,imputed_X_valid, y_train, y_valid ))

Approach 2 Mean Score:
0.08821917808219178


In [32]:
#Approach 3 -- Imputation

X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in col_missing:
    X_train_plus[col + 'was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + 'was_missing'] = X_valid_plus[col].isnull()
     

second_imputer = SimpleImputer()

imputed_X_train_plus = pd.DataFrame(second_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(second_imputer.transform(X_valid_plus))


print('Approach 3 Mean Score')
print(score_dataset(imputed_X_train_plus,imputed_X_valid_plus,y_train, y_valid ))

Approach 3 Mean Score
0.08500978473581212
